In [26]:
import pandas as pd
from sqlalchemy import create_engine, exc
import os
from dotenv import load_dotenv
import sys
from datetime import datetime
import time
import shutil
import tempfile

def log_error(error_msg):
    """Registra erros em um arquivo de log com encoding UTF-8"""
    try:
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        with open("etl_error.log", "a", encoding="utf-8") as f:
            f.write(f"[{timestamp}] {error_msg}\n")
    except Exception as e:
        print(f"Falha ao registrar erro no log: {str(e)}")

def make_local_copy(src_path):
    """Cria uma cópia local temporária do arquivo para evitar problemas com OneDrive"""
    try:
        # Cria um diretório temporário
        temp_dir = tempfile.mkdtemp()
        # Define o caminho do arquivo temporário
        temp_path = os.path.join(temp_dir, os.path.basename(src_path))
        
        # Copia o arquivo
        shutil.copy2(src_path, temp_path)
        print(f"Cópia temporária criada em: {temp_path}")
        return temp_path
    except Exception as e:
        raise Exception(f"Falha ao criar cópia local: {str(e)}")

def clean_dataframe(df):
    """Limpeza robusta dos dados"""
    try:
        # Verifica se existe a coluna UF
        if 'uf' not in df.columns:
            raise ValueError("Coluna 'uf' não encontrada no DataFrame")
            
        # Remove linhas com UF vazio
        initial_count = len(df)
        df = df[df['uf'].notna() & (df['uf'].astype(str).str.strip() != '')]
        cleaned_count = initial_count - len(df)
        print(f"Removidas {cleaned_count} linhas com uf vazio/inválido")
        
        # Remove linhas completamente vazias
        df = df.dropna(how='all')
        
        # Limpeza de strings
        str_cols = df.select_dtypes(include=['object']).columns
        df[str_cols] = df[str_cols].apply(lambda x: x.str.strip() if x.dtype == "object" else x)
        
        return df
    except Exception as e:
        raise ValueError(f"Erro na limpeza dos dados: {str(e)}")

def load_data_to_db():
    try:
        # Configuração inicial
        load_dotenv()
        print("Iniciando processo ETL...")
        
        # Verifica variáveis de ambiente
        required_vars = ["DB_USER", "DB_PASSWORD", "DB_HOST", "DB_NAME"]
        missing_vars = [var for var in required_vars if not os.getenv(var)]
        if missing_vars:
            raise ValueError(f"Variáveis ausentes: {', '.join(missing_vars)}")
        
        # Conexão com o banco
        DATABASE_URL = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}/{os.getenv('DB_NAME')}?sslmode={os.getenv('DB_SSL_MODE', 'require')}"
        engine = create_engine(DATABASE_URL)
        
        # Caminho do arquivo
        excel_path = os.path.join(os.getcwd(), "databases", "database-sec.xlsx")
        print(f"Tentando acessar: {excel_path}")
        
        # Cria cópia local temporária
        temp_excel_path = make_local_copy(excel_path)
        
        try:
            # Processamento do Excel
            with pd.ExcelFile(temp_excel_path) as excel:
                # Verifica planilhas
                if 'in' not in excel.sheet_names:
                    raise ValueError("Planilha 'in' não encontrada")
                
                # Leitura dos dados
                df = pd.read_excel(excel, sheet_name='in')
                print(f"Total de registros lidos: {len(df)}")
                
                # Limpeza
                df = clean_dataframe(df)
                if df.empty:
                    raise ValueError("DataFrame vazio após limpeza")
                
                # Carregamento no banco
                df.to_sql(
                    "dataframe_pmdf",
                    engine,
                    if_exists="replace",
                    index=False,
                    method="multi",
                    chunksize=1000
                )
                print(f"✅ Dados carregados com sucesso! Total: {len(df)} registros")
                
        finally:
            # Remove a cópia temporária
            try:
                os.remove(temp_excel_path)
                os.rmdir(os.path.dirname(temp_excel_path))
            except Exception as e:
                print(f"Aviso: Não foi possível limpar arquivos temporários: {str(e)}")
                
    except Exception as e:
        error_msg = f"ERRO: {str(e)}"
        print(error_msg)
        log_error(error_msg)
        sys.exit(1)

if __name__ == "__main__":
    load_data_to_db()

Iniciando processo ETL...
Tentando acessar: c:\Users\JS982PS\OneDrive - EY\Documents\pmdf-analytcs\databases\database-sec.xlsx
Cópia temporária criada em: C:\Users\JS982PS\AppData\Local\Temp\tmpdyxc0zbh\database-sec.xlsx
Total de registros lidos: 700344
Removidas 687113 linhas com uf vazio/inválido
✅ Dados carregados com sucesso! Total: 13231 registros


In [27]:
display(df)

{'in':          uf      municipio             evento   mês     ano data_referencia  \
 0       NaN            NaN                NaN   NaN     NaN             NaT   
 1       NaN            NaN                NaN   NaN     NaN             NaT   
 2       NaN            NaN                NaN   NaN     NaN             NaT   
 3       NaN            NaN                NaN   NaN     NaN             NaT   
 4       NaN            NaN                NaN   NaN     NaN             NaT   
 ...     ...            ...                ...   ...     ...             ...   
 700339   DF  NÃO INFORMADO  Tráfico de drogas   8.0  2022.0      2022-08-01   
 700340   DF  NÃO INFORMADO  Tráfico de drogas   9.0  2022.0      2022-09-01   
 700341   DF  NÃO INFORMADO  Tráfico de drogas  10.0  2022.0      2022-10-01   
 700342   DF  NÃO INFORMADO  Tráfico de drogas  11.0  2022.0      2022-11-01   
 700343   DF  NÃO INFORMADO  Tráfico de drogas  12.0  2022.0      2022-12-01   
 
        agente arma  feminino  m